In [ ]:
##GET DATA FROM SAGE ARCHIVE##
t1 = UTCDateTime(2023, 8, 11) # Start time
t2 = UTCDateTime(2023, 8, 14) # End time
delta = 1800 # time in seconds, reduce to reduce file size
stas = [2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2313, 2314, 2315, 2316, 
        'G2301', 'G2302', 'G2303', 'G2304', 'G2305', 'G2306', 'G2307', 'G2308', 'G2309', 'G2310', 'G2311',
        'G2312', 'G2313', 'G2314', 'G2315', 'G2316'] 
#station identifier set for both ZE(3C Nodes)[PH5] and ZD(GEM loggers & in-ground sesimic data)[MiniSEED]. 
#Convention is year/site (YR/XX), G(YR/XX) for GEM loggers
##NOTE: ZD is missing G2308 & G2316 entirely, gaps in others
stas = [2302] # station id's actually used for resampling

#Be sure to use the appropriate channel for the data type 
#'fdsnws/dataselect' for MINIseed (GEM and RT130)
#'ph5ws/dataselect' for PH5 (3C node)
DATASELECT = 'http://service.iris.edu/ph5ws/dataselect/1'
c = fdsn.client.Client(
    service_mappings={
        'dataselect': DATASELECT,
    },
)
c.set_credentials('', '')

tNow = t1 #start time
req = [] #UNSURE????
while tNow < t2:
    print(tNow)
    for nowsta in stas:
        # req.append(('ZD', nowsta, '01', 'HDF', tNow, tNow+delta)) for GEM infrasound data in MINIseed
        # req.append(('ZE', nowsta, '', 'GPZ', tNow, tNow+delta)) for SmartSolo 3C node data
        print(nowsta)
        print('Getting data')
        S = c.get_waveforms('ZE', nowsta, '', 'GP1', tNow, tNow+delta)
        print('Resampling')
        S.resample(350) # resampled rate for final analysis. note: remeber what Nyquist f you would like to work with, multiply by (1.1<f<1.5) for this value
        # Recently using 350Hz for 3C data and 120Hz for GEM data
        print('Saving')
        for tr in S:
            filename = 'E:/ActiveResearch/NisquallyRiver_SeismicInfrasound/NodeData/3C/2023/20240720_20240722/{}_{}.mseed'.format(tr.id, tr.stats.starttime.strftime('%Y%m%d%H%M%S')) # data format, tr.id==data directory, time format
            tr.write(filename)
    tNow = tNow + delta